In [402]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv("train.csv")
df = df.drop(["PassengerId","Cabin"], axis = 1)
df = df.replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
honor = []
for i in df["Name"]:
    if "Master" in i :
        honor.append(0)
    elif "Mr" in i :
        honor.append(1)
    elif "Miss" in i :
        honor.append(2)
    elif "Mrs" in i :
        honor.append(3)
    else:
        honor.append("4")
df['honor'] = np.array(honor)
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
df = df.fillna(value = {'Embarked': 1})
df = df.fillna(value = {"Age":-1})

y = df["Survived"].as_matrix()

In [403]:
name_df_train = df.iloc[:,2]
survive_array = df.iloc[:,0]

name_df = name_df_train

name_array = []
name_count = []
name_survive_count = []
name_survive_ratio = []

name_i = 0

for name in name_df:
    name_split = name.split(" ")
    name_split_first = name_split[0].replace(',','')
    if name_split_first in name_array:
        name_count[name_array.index(name_split_first)] = name_count[name_array.index(name_split_first)] + 1
        name_survive_count[name_array.index(name_split_first)] = name_survive_count[name_array.index(name_split_first)] + survive_array[name_i]                            
    else:
        name_array.append(name_split_first)
        name_count.append(1)
        name_survive_count.append(survive_array[name_i])
    name_i = name_i + 1
        
count = 0
for i in name_count:
    if name_count[count] == 1:
        name_survive_ratio.append(0)
    else:
        name_survive_ratio.append(name_survive_count[count]/(name_count[count]))
    count = count + 1
    
survive_ratio_array = []    

for name in name_df:
    name_split = name.split(" ")
    name_split_first = name_split[0].replace(',','')
    survive_ratio_array.append(name_survive_ratio[name_array.index(name_split_first)])

df['familysurviveratio'] = np.array(survive_ratio_array)

In [404]:
a = df.loc[:,["Age","honor"]]
a =np.array(a)
for i in a:
    if i[0] == -1:
        i[0] =df[df["honor"]==i[1]]["Age"].mean()
frame = pd.DataFrame(a, columns=["Age", "honor"])
df = df.drop(["Survived","Age","honor","Name","Ticket","SibSp","Parch"], axis = 1)
df = pd.concat([df, frame], axis=1)

In [405]:
df.head()

,Pclass,Sex,Fare,Embarked,FamilySize,familysurviveratio,Age,honor
0,3,0,7.2500,0.0,2,0.0,22,1
1,1,1,71.2833,1.0,2,0.0,38,1
2,3,1,7.9250,0.0,1,0.0,26,2
3,1,1,53.1000,0.0,2,0.5,35,1
4,3,0,8.0500,0.0,1,0.5,35,1


In [406]:
X= df.as_matrix()

In [407]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [408]:
forest = RandomForestClassifier(min_samples_leaf=3, random_state=0)
forest.fit(X_train, y_train)
print('Train score: {}'.format(forest.score(X_train, y_train)))
print('Test score: {}'.format(forest.score(X_test, y_test)))

Train score: 0.8952095808383234
Test score: 0.8609865470852018


In [411]:
 from sklearn.model_selection import GridSearchCV

 # use a full grid over all parameters
 param_grid = {"max_depth": [2,3, None],
              "n_estimators":[50,100,200,300,400,500],
              "max_features": [1, 3, 8],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 8],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

 forest_grid = GridSearchCV(estimator=RandomForestClassifier(random_state=0),
                 param_grid = param_grid,   
                 scoring="accuracy",  #metrics
                 cv = 3,              #cross-validation
                 n_jobs = 1)          #number of core

 forest_grid.fit(X_train,y_train) #fit

 forest_grid_best = forest_grid.best_estimator_ #best estimator
 print("Best Model Parameter: ",forest_grid.best_params_)

Best Model Parameter:  {'bootstrap': True, 'criterion': 'gini', 'max_depth': None, 'max_features': 8, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 300}


In [412]:
print('Train score: {}'.format(forest_grid.score(X_train, y_train)))
print('Test score: {}'.format(forest_grid.score(X_test, y_test)))

Train score: 0.907185628742515
Test score: 0.8654708520179372


In [399]:
from sklearn.ensemble import GradientBoostingClassifier
gbct = GradientBoostingClassifier(random_state=0, max_depth=3, learning_rate=0.1)
gbct.fit(X_train, y_train)
print('train accuracy: %.3f' % gbct.score(X_train, y_train))
print('test  accuracy: %.3f' % gbct.score(X_test, y_test))

train accuracy: 0.918
test  accuracy: 0.865


In [400]:
import xgboost as xgb
mod = xgb.XGBClassifier()
mod.fit(X_train, y_train)
print('train accuracy: %.3f' % mod.score(X_train, y_train))
print('test  accuracy: %.3f' % mod.score(X_test, y_test))

train accuracy: 0.904
test  accuracy: 0.874


In [375]:
df2 = pd.read_csv("test.csv")
df2 = df2.drop(["PassengerId","Ticket","Cabin"], axis = 1)
df2 = df2.replace("male",0).replace("female",1)
df2 = df2.replace("S",0).replace("C",1).replace("Q",2)
df2["FamilySize"] = df2["SibSp"] + df2["Parch"] + 1
df2 = df2.fillna(value = {'Embarked': 1})
df2 = df2.fillna(value = {"Age":-1})
honor = []
for i in df2["Name"]:
    if "Master" in i :
        honor.append(0)
    elif "Mr" in i :
        honor.append(1)
    elif "Miss" in i :
        honor.append(2)
    elif "Mrs" in i :
        honor.append(3)
    else:
        honor.append("4")
    
df2['honor'] = np.array(honor)
a = df2.loc[:,["Age","honor"]]
a =np.array(a)
for i in a:
    if i[0] == -1:
        i[0] =df2[df2["honor"]==i[1]]["Age"].mean()
frame = pd.DataFrame(a, columns=["Age", "honor"])
df2 = df2.drop(["Age","honor"], axis = 1)
df2 = pd.concat([df2, frame], axis=1)
name_df_test = df2.iloc[:,1]
survive_ratio_test =[]
for name in name_df_test:
    name_split = name.split(" ")
    name_split_first = name_split[0].replace(',','')                                         
    if name_split_first in name_array:
        survive_ratio_test.append(name_survive_ratio[name_array.index(name_split_first)])
    else:
        survive_ratio_test.append(0)
df2= df2.drop(["Name","SibSp","Parch"], axis = 1)        
df2['familysurviveratio'] = np.array(survive_ratio_test)
df2.head()

,Pclass,Sex,Fare,Embarked,FamilySize,Age,honor,familysurviveratio
0,3,0,7.8292,2,1,34.5,1,0.75
1,3,1,7.0000,0,2,47,1,0.00
2,2,0,9.6875,2,1,62,1,0.00
3,3,0,8.6625,0,1,27,1,0.00
4,3,1,12.2875,0,3,22,1,0.00


In [376]:
df2.count()
df2 = df2.fillna(df2.mean())
df2.count()

df2.head()
df = df.loc[:,['Pclass','Sex','Fare','Embarked','FamilySize','familysurviveratio','Age','honor']]
df2 =df2.loc[:,['Pclass','Sex','Fare','Embarked','FamilySize','familysurviveratio','Age','honor']]


In [377]:
df.head()

,Pclass,Sex,Fare,Embarked,FamilySize,familysurviveratio,Age,honor
0,3,0,7.2500,0.0,2,0.0,22,1
1,1,1,71.2833,1.0,2,0.0,38,1
2,3,1,7.9250,0.0,1,0.0,26,2
3,1,1,53.1000,0.0,2,0.5,35,1
4,3,0,8.0500,0.0,1,0.5,35,1


In [378]:
df2.head()

,Pclass,Sex,Fare,Embarked,FamilySize,familysurviveratio,Age,honor
0,3,0,7.8292,2,1,0.75,34.5,1
1,3,1,7.0000,0,2,0.00,47.0,1
2,2,0,9.6875,2,1,0.00,62.0,1
3,3,0,8.6625,0,1,0.00,27.0,1
4,3,1,12.2875,0,3,0.00,22.0,1


In [379]:
df2.head()
x = df2.as_matrix()

In [380]:
mod1 = xgb.XGBClassifier()
mod1.fit(X, y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [381]:
y1 = mod1.predict(x)

In [382]:
a =[i for i in range(892,1310)]
dfnew2 = pd.DataFrame({'PassengerId':a ,'Survived': y1})
dfnew2.to_csv("pred19.csv") 

In [387]:
gbct2 = GradientBoostingClassifier(random_state=0, max_depth=3, learning_rate=0.1)
gbct2.fit(X, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

In [388]:
y2 = gbct2.predict(x)

In [389]:
a =[i for i in range(892,1310)]
dfnew2 = pd.DataFrame({'PassengerId':a ,'Survived': y2})
dfnew2.to_csv("pred20.csv") 

In [413]:
y3 = forest_grid.predict(x)

In [414]:
a =[i for i in range(892,1310)]
dfnew2 = pd.DataFrame({'PassengerId':a ,'Survived': y3})
dfnew2.to_csv("pred21.csv") 